In [22]:
import bs4 as bs
import datetime as dt
import os
import pickle
import requests
import pandas as pd
import yfinance as yf
import pymongo
import string

In [24]:
ticker_to_company_name = pd.read_csv('secwiki_tickers.csv')
ticker_to_company_name['Name'] = ticker_to_company_name['Name'].str.translate(str.maketrans('','', string.punctuation))
ticker_to_company_name['Name'] = ticker_to_company_name['Name'].str.upper()
ticker_to_company_name['Name'] = ticker_to_company_name['Name'].str.strip()
ticker_to_company_name.head()

,Ticker,Name,Sector,Industry,Price,Collection
0,A,AGILENT TECHNOLOGIES INC,Healthcare,Medical Laboratories & Research,WIKI/A,stocks/a
1,AA,ALCOA INC,Basic Materials,Aluminum,WIKI/AA,stocks/aa
2,AAME,ATLANTIC AMERICAN CORP,Financial,Life Insurance,NaN,stocks/aame
3,AAN,AARONS INC,Services,Rental & Leasing Services,WIKI/AAN,stocks/aan
4,AAON,AAON INC,Industrial Goods,General Building Materials,WIKI/AAON,stocks/aaon


In [66]:
x = ticker_to_company_name.loc[ticker_to_company_name['Name'] == 'APPLE INC']#['Ticker'].iloc[0]
if ticker_to_company_name.loc[ticker_to_company_name['Name'] == 'APPLE IN'].empty:
    print('yes')

yes


In [28]:
def find_ticker_data(company_name, ticker_to_company_df):
    try:
        company_name = company_name.translate(str.maketrans('','', string.punctuation))
        company_name = company_name.upper()
        company_name = company_name.strip()
        ticker = ticker_to_company_df.loc[ticker_to_company_df['Name'] == company_name]['Ticker'].iloc[0]
        company_sector = ticker_to_company_df.loc[ticker_to_company_df['Name'] == company_name]['Sector'].iloc[0]
        name = ticker_to_company_df.loc[ticker_to_company_df['Name'] == company_name]['Name'].iloc[0]
    except:
        print('This is not a valid company name.')
        return 'This is not a valid company name.'
    stock = yf.Ticker(ticker)
    stock_data=stock.history(period="1mo",actions=False,auto_adjust=False)
    stock_data['Symbol']=ticker
    stock_data['Sector']= company_sector
    stock_data['Company Name'] = name
    stock_data = stock_data[['Close', 'Symbol', 'Sector', 'Company Name']]
    stock_data.reset_index
    return stock_data

In [58]:
df2 = find_ticker_data('YAHOO INC', ticker_to_company_name)

- YHOO: No data found for this date range, symbol may be delisted


In [59]:
df2

,Close,Symbol,Sector,Company Name
Date,,,,


In [54]:
df4 = up_down(df2)
df4.reset_index(inplace = True)
df4.sort_values(by = 'Date', inplace = True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
df4

,Date,Close,Symbol,Sector,Company Name,Price Change
0,2020-01-07,298.39,AAPL,Consumer Goods,APPLE INC,down
1,2020-01-08,303.19,AAPL,Consumer Goods,APPLE INC,up
2,2020-01-09,309.63,AAPL,Consumer Goods,APPLE INC,up
3,2020-01-10,310.33,AAPL,Consumer Goods,APPLE INC,up
4,2020-01-13,316.96,AAPL,Consumer Goods,APPLE INC,up
5,2020-01-14,312.68,AAPL,Consumer Goods,APPLE INC,down
6,2020-01-15,311.34,AAPL,Consumer Goods,APPLE INC,down


In [50]:
def up_down(df):
    day_change = []
    for i in range(len(df)):
        if i == 0:
            day_change.append(None)
            continue
        today = df.iloc[i,0]
        yesterday = df.iloc[i-1,0]
        if today >= yesterday:
            day_change.append('up')
        else:
            day_change.append('down')
    df['Price Change'] = day_change
    df.reset_index(inplace = True)
    return df.iloc[-7:]
        
        

In [ ]:
df_ = []
count=0

# loop through ticker list and pull historic market data from yahoo Finance
for ticker in ticker:
    stock = yf.Ticker(ticker)
    stock_data=stock.history(period="1mo",actions=False,auto_adjust=False)
    # map  ticker symbol to sector and add as columns to dataframe
    stock_data['Symbol']=ticker
    stock_data['Sector']=stock_data['Symbol'].map(sector_map)
    df_.append(stock_data)
    
    #continue appending dataframe to a list (df_) and pop off the last dataframe. df_[0] is now the ultimate dataframe with al stockdata.
    if (len(df_)>1):
        df_[0] = df_[0].append(df_[-1])
        df_.pop(-1)

# rename the dataframe. final_data is the ultimate dataframe with stockdata for S&P 500 for 1 month period
final_data=df_[0]
final_data